# The following notebook uses the dataframe on which tracking has been performed and uses the spots coordinates to extract activity from alternate channels. 

In [1]:
from os import path
import pandas as pd
from IPython.display import display
import numpy as np 
import sys 
import time 
import zarr

sys.path.append('../src/')

from extract_pixel_data import Extractor

#plt.rcParams["font.family"] = ""

In [2]:
#provide the file path of the tracking dataframe returned from notebook 04 
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output/track_df_c3_cleaned.pkl"
track_df = pd.read_pickle(file_path)

In [3]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,0,0
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,1,1
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,2,2
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,3,3
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,198325,198325
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,198326,198326
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,198327,198327
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,195783,195783


In [4]:
# read the zarr file which contains the data of all three channels 
z = zarr.open('/Users/apple/Desktop/Akamatsu_Lab/Data/Zarr_file_new/all_channels_data', mode='r')

In [5]:
z.info

Type,zarr.core.Array
Data type,uint16
Shape,"(130, 3, 116, 2052, 340)"
Chunk shape,"(1, 1, 116, 2052, 340)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,63126086400 (58.8G)
No. bytes stored,13548767504 (12.6G)
Storage ratio,4.7


In [6]:
track_df['frame'].nunique()

130

## In the below cell the Extractor object is created 

In [7]:
# Create an extractor object with required parameters to properly run the methods in next steps 
# radii is the expected radius of the CME sites (will be used for calculations which need fixed radius) 
# radii_col_name is a list of the variable sigma estimated by gaussian fitting (note: the convention for the list is z,y,x)
# n_jobs is the cores to be used. (parallel processing for gaussian fitting on the other two channels could be done to speed up the process)
extractor = Extractor(z, dataframe = track_df, radii=[4,2,2], frame_col_name = 'frame', 
                      radi_col_name = ['sigma_z', 'sigma_y', 'sigma_x'], n_jobs =2)

# Extract Information for Channel 3 (Clathrin)

## Extracting voxel sum

In [8]:
# pass the channel for which voxel sum is needed and the coordinates around which voxel sum is supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
# channel number is to passed according to whichever channel we want to extract the data for
start_time = time.time()
voxel_sum_array, _ = extractor.voxel_sum_fixed_bd(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 3)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [9]:
voxel_sum_array_3, _, adj_voxel_sum_3 = extractor.voxel_sum_fixed_background(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 3)

current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [10]:
# pass the channel for which pixel values are needed and the coordinates around which values are supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 3)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [11]:
max_loc = np.array(max_loc)

In [12]:
track_df['c3_mean_amp'] = mean
track_df['c3_voxel_sum'] = voxel_sum_array_3
track_df['c3_voxel_sum_adjusted'] = adj_voxel_sum_3 
track_df['c3_peak_amp'] = maximum 
track_df['c3_peak_x'] = max_loc[:,2]
track_df['c3_peak_y'] = max_loc[:,1]
track_df['c3_peak_z'] = max_loc[:,0]

In [13]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id,c3_mean_amp,c3_voxel_sum,c3_voxel_sum_adjusted,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,0,0,134.866667,12927,69.153846,180,20,1814,2
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,1,1,143.097143,15894,-133.454545,193,25,1805,4
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,2,2,132.306667,14697,914.000000,187,50,570,4
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,3,3,141.000000,15905,60.818182,181,134,740,4
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,4,4,133.453061,14972,13.000000,183,39,1125,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,198325,198325,189.560000,23348,964.979592,243,233,1923,110
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,198326,198326,245.466667,30226,778.163265,309,260,1271,109
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,198327,198327,156.588571,19902,990.653061,197,258,1340,110
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,195783,195783,151.104000,19043,189.775510,179,277,1326,110


# Extract information for Channel 2 

In [14]:
# pass the channel for which pixel values are needed and the coordinates around which values are supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 2)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [15]:
max_loc = np.array(max_loc)

## Finding peak pixel for Channel 2 as not sure about the offset between Channel 3 and Channel 2
**Reason for offset is being shot from different cameras**

In [16]:
track_df['c2_amp'] = mean
track_df['c2_peak'] = maximum
track_df['c2_peak_x'] = max_loc[:,2]
track_df['c2_peak_y'] = max_loc[:,1]
track_df['c2_peak_z'] = max_loc[:,0]

In [17]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_voxel_sum_adjusted,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,69.153846,180,20,1814,2,112.226667,131,21,1813,5
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,-133.454545,193,25,1805,4,114.851429,142,25,1806,5
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,914.000000,187,50,570,4,106.986667,123,50,570,5
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,60.818182,181,134,740,4,120.811429,148,135,739,5
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,13.000000,183,39,1125,5,116.130612,137,40,1126,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,...,964.979592,243,233,1923,110,133.184000,187,233,1923,110
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,...,778.163265,309,260,1271,109,139.226667,185,260,1273,108
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,...,990.653061,197,258,1340,110,127.994286,164,262,1339,109
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,...,189.775510,179,277,1326,110,120.080000,146,277,1326,109


## Finding mean value around the peak pixel value for channel 2 

In [18]:
start_time = time.time()
col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x']
peak_mean,maxima,_,_,_ = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, 
                                                     channel = 2)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [19]:
track_df['c2_peak_mean'] = peak_mean

In [20]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_peak_amp,c3_peak_x,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,180,20,1814,2,112.226667,131,21,1813,5,119.653333
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,193,25,1805,4,114.851429,142,25,1806,5,115.931429
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,187,50,570,4,106.986667,123,50,570,5,108.906667
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,181,134,740,4,120.811429,148,135,739,5,119.777143
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,183,39,1125,5,116.130612,137,40,1126,5,117.432653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,...,243,233,1923,110,133.184000,187,233,1923,110,159.944000
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,...,309,260,1271,109,139.226667,185,260,1273,108,171.920000
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,...,197,258,1340,110,127.994286,164,262,1339,109,130.994286
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,...,179,277,1326,110,120.080000,146,277,1326,109,121.992000


## Finding voxel sum around peak for channel 2 

In [22]:
# pass the channel for which voxel sum is needed and the coordinates around which voxel sum is supposed to be calculated
# convention for coords [z,y,x] 
# convention for channel is 1 for channel 1, 2 for channel 2 and so on 
start_time = time.time()
voxel_sum_array_2, _, adj_voxel_sum_array_2 = extractor.voxel_sum_fixed_background(center_col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                       channel = 2)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [23]:
#calculated around the peak value coordinates
track_df['c2_voxel_sum'] = voxel_sum_array_2
track_df['c2_voxel_sum_adjusted'] = adj_voxel_sum_array_2

In [24]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_peak_y,c3_peak_z,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean,c2_voxel_sum,c2_voxel_sum_adjusted
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,1814,2,112.226667,131,21,1813,5,119.653333,15009,-145.938776
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,1805,4,114.851429,142,25,1806,5,115.931429,14668,-284.489796
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,570,4,106.986667,123,50,570,5,108.906667,13769,-69.367347
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,740,4,120.811429,148,135,739,5,119.777143,15401,212.102041
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,1125,5,116.130612,137,40,1126,5,117.432653,14998,139.551020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,...,1923,110,133.184000,187,233,1923,110,159.944000,19572,1046.206897
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,...,1271,109,139.226667,185,260,1273,108,171.920000,20844,573.103448
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,...,1340,110,127.994286,164,262,1339,109,130.994286,16601,410.655172
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,...,1326,110,120.080000,146,277,1326,109,121.992000,15447,18.586207


### Gaussian Fitting for Channel 2 around peak values 

In [ ]:
#Calculating the gaussian fitting estimates around peak coords for channel 2 
#expected sigma value needed for gaussian fitting 
#set all frames to False for processing limited frames 
#max_frames determines the frames to be processed if all_frames is false
start_time = time.time()
channel2_gaussians_df = extractor.run_parallel_frame_processing(expected_sigma = [4,2,2], 
                                        center_col_name = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'], 
                                       dist_between_spots = 10 , channel = 2,  max_frames =  2, all_frames = False)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

In [ ]:
track_df['c2_gaussian_amp'] = channel2_gaussians_df['amplitude']
track_df['c2_mu_x'] = channel2_gaussians_df['mu_x']
track_df['c2_mu_y'] = channel2_gaussians_df['mu_y']
track_df['c2_mu_z'] = channel2_gaussians_df['mu_z']
track_df['c2_sigma_x'] = channel2_gaussians_df['sigma_x']
track_df['c2_sigma_y'] = channel2_gaussians_df['sigma_y']
track_df['c2_sigma_z'] = channel2_gaussians_df['sigma_z']

In [ ]:
track_df

# Extract information for channel 1

In [25]:
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
mean,maximum,minimum,pixel_values,max_loc = extractor.extract_pixels_data_variable_bd(center_col_names = col_names, channel = 1)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [26]:
max_loc = np.array(max_loc)

In [27]:
track_df['c1_amp'] = mean
track_df['c1_peak'] = maximum
track_df['c1_peak_x'] = max_loc[:,2]
track_df['c1_peak_y'] = max_loc[:,1]
track_df['c1_peak_z'] = max_loc[:,0]

In [28]:
voxel_sum_array_1, _, adj_voxel_sum_array_1 = extractor.voxel_sum_fixed_background(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 1)


current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [29]:
#calculated around the peak value coordinates
track_df['c1_voxel_sum'] = voxel_sum_array_1
track_df['c1_voxel_sum_adjusted'] = adj_voxel_sum_array_1

In [39]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c2_peak_mean,c2_voxel_sum,c2_voxel_sum_adjusted,c1_amp,c1_peak,c1_peak_x,c1_peak_y,c1_peak_z,c1_voxel_sum,c1_voxel_sum_adjusted
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,119.653333,15009,-145.938776,134.720000,156,19,1813,5,13216,-118.153846
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,115.931429,14668,-284.489796,139.565714,167,24,1806,5,15629,-136.909091
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,108.906667,13769,-69.367347,118.173333,138,50,568,4,13305,199.090909
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,119.777143,15401,212.102041,164.662857,217,132,738,7,18473,-691.727273
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,117.432653,14998,139.551020,150.069388,199,40,1124,7,16814,-481.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,...,159.944000,19572,1046.206897,381.552000,604,233,1921,108,47229,2092.020408
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,...,171.920000,20844,573.103448,696.546667,992,260,1272,109,85970,2900.612245
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,...,130.994286,16601,410.655172,291.742857,366,260,1340,111,35905,2095.367347
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,...,121.992000,15447,18.586207,242.688000,302,274,1326,112,30663,342.020408


In [40]:
track_df.shape

(515935, 34)

### Perform Guassian Fitting for channel 1

In [ ]:
start_time = time.time()
channel1_gaussians_df = extractor.run_parallel_frame_processing(expected_sigma = [4,2,2], 
                                        center_col_name = ['c1_peak_z', 'c1_peak_y', 'c1_peak_x'], 
                                       dist_between_spots = 10, channel = 1,  max_frames =  2, all_frames = False)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

In [ ]:
channel1_gaussians_df

In [ ]:
track_df['c1_gaussian_amp'] = channel1_gaussians_df['amplitude']
track_df['c1_mu_x'] = channel1_gaussians_df['mu_x']
track_df['c1_mu_y'] = channel1_gaussians_df['mu_y']
track_df['c1_mu_z'] = channel1_gaussians_df['mu_z']
track_df['c1_sigma_x'] = channel1_gaussians_df['sigma_x']
track_df['c1_sigma_y'] = channel1_gaussians_df['sigma_y']
track_df['c1_sigma_z'] = channel1_gaussians_df['sigma_z']

In [ ]:
track_df

In [41]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output_zarr/track_df_cleaned_final_full.pkl"
track_df.to_pickle(file_path)

# To test the following functions change the markdown files to code files 
1. Voxel sum using the variable sigma/radi values directly inferred from the dataset 
2. Extracting pixel values mean,max etc using fixed radi
3. Performing Gaussian Fitting on peak coords for single frame

### Function 1 (Voxel Sum)
start_time = time.time()
voxel_sum_array_variable, _ = extractor.voxel_sum_variable_bd(center_col_names = ['mu_z', 'mu_y', 'mu_x'], channel = 3)
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

### Function 2 (Extracting Pixel Values)
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radii = [4, 2, 2]
mean_v,maximum_v,minimum_v,pixel_values_v,max_loc_v = extractor.extract_pixels_data_fixed_bd(center_col_names = col_names, channel = 3)

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

### Function 3 (Gaussian fitting on one frame for one channel)
df = extractor.gaussian_fitting_single_frame(expected_sigma = [4,2,2], 
                              center_col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                      frame = 0, channel = 2, dist_between_spots = 10)